notebook being completely reworded

In [21]:
%reload_ext autoreload
%autoreload 2

In [2]:
import tempfile

import pandas as pd

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from elections import constants
from elections.scrapers.psd_scraper import PsdScrapper

In [4]:
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.tsv') as temp_file:
    df = pd.read_csv(constants.POLITICAL_PATHS["psd"] / "proposals.tsv", sep="\t")[["category_pt", "proposal_pt"]]
    df.to_csv(temp_file.name, index=False, header=False, sep="\t")

    loader = TextLoader(temp_file.name)

In [6]:
text = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)

texts = text_splitter.create_documents([text[0].page_content])

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

tt = HuggingFaceEmbeddings().embed_documents([text.page_content for text in texts])

In [8]:
import numpy as np

np.array(tt).shape


(203, 768)

In [9]:

tt1 = HuggingFaceEmbeddings(model_name='ricardo-filho/bert-base-portuguese-cased-nli-assin-2').embed_documents([text.page_content for text in texts])

In [10]:
from langchain.indexes import VectorstoreIndexCreator

pt_embeddings = HuggingFaceEmbeddings(model_name='ricardo-filho/bert-base-portuguese-cased-nli-assin-2')
db = VectorstoreIndexCreator(embedding= pt_embeddings, text_splitter=text_splitter).from_loaders([loader])

In [11]:
db

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x2ca53f850>)

In [14]:
tt5 = db.vectorstore.similarity_search("quais os desafios com o SNS?", k=5)
tt5

[Document(page_content='do SNS, bem como a medidas estruturais com impacto no longo prazo. O acesso atempado e universal a', metadata={'source': '/var/folders/q3/m5wwnysd3kx0n1_6yvsrqghh0000gn/T/tmp2zcuj21w.tsv'}),
 Document(page_content='revisao constitucional\tIntrodução da referência de que o SNS deve cobrir as necessidades de toda a', metadata={'source': '/var/folders/q3/m5wwnysd3kx0n1_6yvsrqghh0000gn/T/tmp2zcuj21w.tsv'}),
 Document(page_content='constituir família. As políticas para remover os obstáculos à natalidade nos jovens casais só se', metadata={'source': '/var/folders/q3/m5wwnysd3kx0n1_6yvsrqghh0000gn/T/tmp2zcuj21w.tsv'}),
 Document(page_content='os problemas estruturais dos jovens, como os baixos salários e a dificuldade em constituir família.', metadata={'source': '/var/folders/q3/m5wwnysd3kx0n1_6yvsrqghh0000gn/T/tmp2zcuj21w.tsv'}),
 Document(page_content='aos idosos. Estender ao setor privado essa comparticipação quando a rede pública e social não', metadata={'source': 

In [15]:
df

,category_pt,proposal_pt
0,fiscalidade,Portugal tem um problema muito sério de asfixi...
1,fiscalidade,Redução generalizada das taxas marginais nos p...
2,fiscalidade,Os jovens até aos 35 anos terão um regime fisc...
3,fiscalidade,"Para evitar o aumento da carga fiscal, o PSD p..."
4,fiscalidade,"Alteração legal que permite, em caso de excess..."
...,...,...
59,saude,Garantir que todos os cidadãos que não tenham ...
60,saude,Contratualizar diretamente com os setores públ...
61,saude,É fundamental reforçar a Rede de Cuidados Cont...
62,saude,Deve ser atribuída uma contribuição financeira...
